# Required Library

In [1]:
!pip install spacy
!pip install jsonlines

In [2]:
import os, sys
import re
from google.colab import drive
import spacy
import random
import re
from spacy.util import minibatch, compounding
from spacy.scorer import Scorer
from spacy.gold import GoldParse
from spacy import displacy
import pandas as pd
import jsonlines
import random
import math
from pathlib import Path

In [ ]:
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
ssss


ValueError: ignored

# Named Entity Recognition


## NER dataset




In [ ]:
'''
Dataset yang diperlukan memiliki format seperti berikut yang merupakan list of tuples.
[("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]})]
'''
dataset = jsonlines.open('/drive/My Drive/9. NLP 101 & NLP modelling using spaCy/dataset/data-training.json')
dataset = [(data['text'], {"entities": data['labels']}) for data in dataset if len(data['labels']) != 0]

In [ ]:
dataset

In [ ]:
split_length = math.ceil(len(dataset) * 0.85)
random.shuffle(dataset)
TRAIN_DATA = dataset[:split_length]
TEST_DATA = dataset[split_length:]

##**NER Trainer** 

In [ ]:
 path_MODEL = os.path.abspath('/drive/My Drive/9. NLP 101 & NLP modelling using spaCy/model/id_maslahah_ner')
blank = True
n_iter = 100

if blank:
    nlp = spacy.blank("id")  # create blank Language class
    print("Created blank 'id' ner model")
else:
    nlp = spacy.load(path_MODEL)  # load existing spaCy model
    print("Loaded model '%s'" % model)

# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)

# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")

# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
  optimizer = nlp.begin_training()
  for itn in range(n_iter):
      random.shuffle(TRAIN_DATA)
      losses = {}
      # batch up the examples using spaCy's minibatch
      batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
      for batch in batches:
          texts, annotations = zip(*batch)
          nlp.update(
              texts,  # batch of texts
              annotations,  # batch of annotations
              drop=0.4,  # dropout - make it harder to memorise data
              sgd=optimizer,
              losses=losses)
      print("Losses", losses)

scorer_test = Scorer()
for text, _ann_ in TEST_DATA:
    doc = nlp(text)
    doc_gold_text = nlp.make_doc(text)
    gold = GoldParse(doc_gold_text, entities=_ann_['entities'])
    try:
        scorer_test.score(doc, gold)
    except:
        continue

output_dir = Path(path_MODEL)
if not output_dir.exists():
  output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)
print("{}\n\n".format(scorer_test.scores))

# test model
model = spacy.load(path_MODEL)
text = "Joko Widodo adalah Presiden Indonesia"
doc = model(text)
for ent in doc.ents:
  print (f'{ent.text} is {ent.label_}')

# test model menggunakan displacy
# displacy.serve(doc, style="ent")

In [ ]:
model = spacy.load(path_MODEL)
text = "sya membayar 45 rb"
doc = model(text)
for ent in doc.ents:
  print (f'{ent.text} is {ent.label_}')

# Sentiment Analysis
Di dalam spacy sendiri, kita bisa menggunakan custom model ketika melakukan training model yang berbentuk dokumen klasifikasi. Pada kali ini contoh yang diambil adalah menggunakan model dengan arsitektur nya adalah simple-cnn dan menggunakan 
TextCategorizer component. Data yang dipakai saya dapatkan dari Github, berupa dataset review produk di website Female Daily.


## Sentiment Dataset

In [3]:
import pandas as pd

In [4]:
url = 'https://raw.githubusercontent.com/chlaudiah/Sentiment-Classification-FD-Reviews/master/dataset.csv'
df = pd.read_csv(url, error_bad_lines=False)
df

,ID Data,Nama Produk,Jenis Kulit,Review Produk,Rating,Label
0,A001,Cetaphil: Gentle Skin Cleanser,Oily,Aku suka banget sama ini. Cuma ini cleanser ya...,5,Positive
1,A002,Cetaphil: Gentle Skin Cleanser,Oily,Walaupun produknya mengklaim dapat digunakan s...,5,Positive
2,A003,Cetaphil: Gentle Skin Cleanser,Dry,Thanks God for this product. Produk ini udah p...,5,Positive
3,A004,Cetaphil: Gentle Skin Cleanser,Oily,Another favorite product from Cetaphil. Seneng...,5,Positive
4,A005,Cetaphil: Gentle Skin Cleanser,Oily,awalnya kaget karena cleanser ini enggak berbu...,5,Positive
...,...,...,...,...,...,...
495,A496,Acnes: Creamy Wash,Dry,Aku beli ini karna baca blog female daily 3 cl...,1,negative
496,A497,Acnes: Creamy Wash,Dry,"aku pake ini udh ngabisin 1 botol, saat aku je...",1,negative
497,A498,Acnes: Creamy Wash,Dry,Facial Wash yg aku beli pas awal2 mulai jerawa...,1,negative
498,A499,Acnes: Creamy Wash,Dry,Aku pernah beli ini dengan harapan bisa bantu ...,1,negative


In [5]:
def load_data_sentiment(ratio=0.85):
  split = math.ceil(len(df['Review Produk']) * 0.85)
  texts = list(df['Review Produk'])
  cats = [{"POSITIVE": True, "NEGATIVE": False} if label.lower() == "positive" else {"POSITIVE": False, "NEGATIVE": True} for label in list(df['Label'])]
  return (texts[:split], cats[:split]), (texts[split:], cats[split:])

In [6]:
(train_texts, train_cats), (dev_texts, dev_cats) = load_data_sentiment(ratio=0.85)
train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))


In [ ]:
train_data

## Evaluation Function

In [7]:
def evaluate(tokenizer, textcat, texts, cats):
  docs = (tokenizer(text) for text in texts)
  tp = 0.0  # True positives
  fp = 1e-8  # False positives
  fn = 1e-8  # False negatives
  tn = 0.0  # True negatives
  for i, doc in enumerate(textcat.pipe(docs)):
      gold = cats[i]
      for label, score in doc.cats.items():
          if label not in gold:
              continue
          if label == "NEGATIVE":
              continue
          if score >= 0.5 and gold[label] >= 0.5:
              tp += 1.0
          elif score >= 0.5 and gold[label] < 0.5:
              fp += 1.0
          elif score < 0.5 and gold[label] < 0.5:
              tn += 1
          elif score < 0.5 and gold[label] >= 0.5:
              fn += 1
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  if (precision + recall) == 0:
      f_score = 0.0
  else:
      f_score = 2 * (precision * recall) / (precision + recall)
  return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}

## Sentiment Trainer

In [8]:
path_MODEL = os.path.abspath('/content/sample_data')
blank = True
n_iter = 75

if blank:
    nlp = spacy.blank("id")  # create blank Language class
    print("Created blank 'id' sentiment model")
else:
    nlp = spacy.load(path_MODEL)  # load existing spaCy model
    print("Loaded model '%s'" % model)

# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe(
        "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"}
    )
    nlp.add_pipe(textcat, last=True)
# otherwise, get it, so we can add labels to it
else:
    textcat = nlp.get_pipe("textcat")

# add label to text classifier
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")

(train_texts, train_cats), (dev_texts, dev_cats) = load_data_sentiment(ratio=0.85)

print("Using examples ({} training, {} evaluation)".format(len(train_texts), len(dev_texts)))
train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))

# get names of other pipes to disable them during training
pipe_exceptions = ["textcat", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
    batch_sizes = compounding(4.0, 32.0, 1.001)
    for i in range(n_iter):
        losses = {}
        # batch up the examples using spaCy's minibatch
        random.shuffle(train_data)
        batches = minibatch(train_data, size=batch_sizes)
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print(
            "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
                losses["textcat"],
                scores["textcat_p"],
                scores["textcat_r"],
                scores["textcat_f"],
            )
        )

output_dir = Path(path_MODEL)
if not output_dir.exists():
  output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)


# test the trained model
test_text = "Aku suka banget sama ini. Cuma ini cleanser yang cocok buat aku. Aku udah sering banget gonta-ganti cleanser karena wajahku yang penuh jerawat. Aku udah coba yang low-end dan high-end sekalipun tapi tetep gak ada yang sebagus ini. Karena produknya gentle jadi gak bikin iritasi diwajah, kebanyakan produk cleanser lain yang harsh untuk wajah justru dapat menyebabkan iritasi sehingga munculah jerawat. Very recommended!"
doc = nlp(test_text)
print(test_text, doc.cats)

Created blank 'id' sentiment model
Using examples (425 training, 75 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
2.954	0.794	0.794	0.794
1.335	0.829	0.853	0.841
0.236	0.818	0.794	0.806
0.055	0.862	0.735	0.794
0.018	0.867	0.765	0.812
0.002	0.875	0.824	0.848
0.000	0.875	0.824	0.848
0.000	0.882	0.882	0.882
0.000	0.875	0.824	0.848
0.000	0.882	0.882	0.882
0.000	0.882	0.882	0.882
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.879	0.853	0.866
0.000	0.879	0.853	0.866
0.000	0.879	0.853	0.866
0.000	0.879	0.853	0.866
0.000	0.879	0.853	0.866
0.000	0.882	0.882	0.882
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.857	0.882	0.870
0.000	0.85

In [ ]:
nlp = spacy.load(os.path.abspath('/drive/My Drive/9. NLP 101 & NLP modelling using spaCy/model/id_maslahah_sentiment'))
text = list(df['Review Produk']) if str(incom) != 'nan'
predict = [nlp(data).cats.items() for data in text]

In [ ]:
predict_result = pd.DataFrame(
    {'text': text,
     'prediction': predict
    })

In [ ]:
predict_result